<h1><center>Laboratorio 5: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Tomás Reyes Oyarzún


### **Link de repositorio de GitHub:** [Repositorio - TR](https://github.com/TomiReyes/MDS7202-TR)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Jueves a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
#!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Inserte su código aquí
df_retail = pd.read_pickle("online_retail_data.pickle")
df_retail.head()

### 1. Función para explorar características [5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [3 puntos]
- Imprima un conteo de datos nulos por variable [2 puntos]

NOTA: Para generar los gráficos es **OBLIGATORIO** el uso de plotly.

**Respuesta:**

In [3]:
def explore_data(dataframe_in):
    price_graph = px.histogram(dataframe_in, x="Price", title="Histograma de precios")
    price_graph.show()

    quantity_graph = px.histogram(
        dataframe_in, x="Quantity", title="Histograma de cantidad"
    )
    quantity_graph.show()

    print("Conteo de datos nulos:")
    print(dataframe_in.isnull().sum())

In [ ]:
explore_data(df_retail)

### 2. Eliminando outliers [10 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

Hint: tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).
>**Nota: No modificar el método set_output de la clase IQR**

**Respuesta:**

In [5]:
class IQR(BaseEstimator, TransformerMixin):
    def __init__(self, lambd):
        self.lambd = lambd

    def fit(self, X, y=None):
        self.Q1 = X.quantile(0.25)
        self.Q3 = X.quantile(0.75)
        self.IQR = self.Q3 - self.Q1
        return self

    def transform(self, X):
        lim_inf = self.Q1 - self.lambd * self.IQR
        lim_sup = self.Q3 + self.lambd * self.IQR

        X_filtered = X[~((X < lim_inf) | (X > lim_sup)).any(axis=1)]

        return X_filtered

    def set_output(self, transform="default"):
        return self

#### 2.2 Creación del Pipeline [5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [1 punto]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [1 punto]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [1 punto]


HINT: El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.
>**Nota: Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.**

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
numerical_columns = ["Price", "Quantity"]
categorical_columns = ["Invoice", "StockCode", "Customer ID", "Country"]

numeric_transformations = Pipeline(steps=[("iqr", IQR(lambd=10))])

column_transformer = ColumnTransformer(
    transformers=[
        ("numerical", numeric_transformations, numerical_columns),
        ("categorical", "passthrough", categorical_columns),
    ],
    verbose_feature_names_out=False,
)

column_transformer.set_output(transform="pandas")

df_iqr = column_transformer.fit_transform(df_retail)

explore_data(df_retail)
explore_data(df_iqr)

*Reporte los cambios observados aquí.*

Los gráficos son más interpretables por como se ven los datos. Al aumentar el valor de lambda el límite de los datos aumenta por lo que se consideran más casos, además de que genera menos valores nulos por considerar más datos, para el caso de lambda = 1 hay más de 100.000 datos nulos para Price y Quantity, en cambio, con lambda = 10 hay un poco más de 10.000 datos nulos. 

### 3. Agregando un imputer al pipeline [10 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [1 punto]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [1 punto]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [1 punto]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [2 puntos]

>**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [7]:
categoric_transformations = Pipeline(
    steps=[
        (
            "mode_imputer",
            SimpleImputer(strategy="most_frequent"),
        )
    ]
)

numeric_transformations = Pipeline(
    steps=[
        ("iqr", IQR(lambd=2)),
        ("mean_imputer", SimpleImputer(strategy="mean")),
    ]
)

In [8]:
numerical_columns = ["Price", "Quantity"]
categorical_columns = ["Invoice", "StockCode", "Customer ID", "Country", "InvoiceDate"]

column_transformer = ColumnTransformer(
    transformers=[
        ("numerical", numeric_transformations, numerical_columns),
        ("categorical", categoric_transformations, categorical_columns),
    ],
    verbose_feature_names_out=False,
)

column_transformer.set_output(transform="pandas")

df_mean_imputer = column_transformer.fit_transform(df_retail)

In [ ]:
explore_data(df_mean_imputer)
explore_data(df_iqr)


Para el caso del df_mean_imputer, el precio va de 0 a 8 y la cantidad va de 0 a 25. Para el df_iqr, el precio va de 0 a 30 y la cantidad va de 0 a 120. Por lo tanto, para el caso de mean_imputer los datos están más uniformes dentro de los límites que son menores, mientras que para el caso de iqr, se concentran en valores menores porque el límite superior es mayor. Esto considerando un valor de lambda = 2. 

In [10]:
numeric_transformations_knn = Pipeline(
    steps=[
        ("iqr", IQR(lambd=2)),
        ("knn_imputer", KNNImputer(n_neighbors=3)),
    ]
)

column_transformer_knn = ColumnTransformer(
    transformers=[
        ("numerical", numeric_transformations_knn, numerical_columns),
        ("categorical", categoric_transformations, categorical_columns),
    ],
    verbose_feature_names_out=False,
)

column_transformer_knn.set_output(transform="pandas")

df_knn_imputer = column_transformer_knn.fit_transform(df_retail)

In [ ]:
explore_data(df_knn_imputer)
explore_data(df_iqr)

Para el caso de df_knn_imputer genera límites superiores menores que el caso de de_iqr. Por lo que dentro de los límites los datos son repartidos de manera más uniforme. En el caso de iqr, los datos se concentran en los valores menores para las 2 variables.

In [ ]:
explore_data(df_mean_imputer)
explore_data(df_knn_imputer)

*Escriba su respuesta aquí*

Sin saber si está bien implementado todo el código anterior, ambos casos dieron el mismo resultado, por lo que el mejor modelo es el mean_imputer porque el tiempo de ejecución es menor. 

### 4. Creación de nuevas features [20 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [10 puntos]
(2 puntos por cada custom feature)

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [13]:
def custom_features(dataframe_in):
    dataframe_in["InvoiceDate"] = pd.to_datetime(dataframe_in["InvoiceDate"])

    fechas_clientes = dataframe_in.groupby("Customer ID")["InvoiceDate"]
    primer_visita = fechas_clientes.transform("min")
    ultima_visita = fechas_clientes.transform("max")

    dataframe_in["Length"] = (ultima_visita - primer_visita).dt.days

    hoy = dataframe_in["InvoiceDate"].max()
    dataframe_in["Recency"] = (hoy - ultima_visita).dt.days

    dataframe_in["Frequency"] = fechas_clientes.transform("count")

    dataframe_in["Monetary"] = dataframe_in.groupby("Customer ID")[
        "Quantity"
    ].transform("mean")

    df0 = dataframe_in.sort_values(["Customer ID", "InvoiceDate"])
    df0["Next_Date"] = df0.groupby("Customer ID")["InvoiceDate"].shift(-1)
    df0["IVT"] = (df0["Next_Date"] - df0["InvoiceDate"]).dt.days
    periodicity = df0.groupby("Customer ID")["IVT"].std().fillna(0)

    df_final = dataframe_in.drop_duplicates("Customer ID").copy()
    df_final["Periodicity"] = df_final["Customer ID"].map(periodicity)

    df_out = df_final[
        ["Customer ID", "Length", "Recency", "Frequency", "Monetary", "Periodicity"]
    ]

    return df_out

In [ ]:
df_test = custom_features(df_retail)
df_test.head()

#### 4.2 Agregando las custom features [10 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [1 punto]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [1 punto]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [3 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta**

In [15]:
def lrmfp_features(df):
    df_lrmfp = custom_features(df)
    columnas = df_lrmfp.columns.difference(df.columns).to_list() + ["Customer ID"]

    df_lrmfp = df_lrmfp[columnas]
    df_merge = df.merge(df_lrmfp, on="Customer ID", how="left")
    return df_merge


retail_pipeline = Pipeline(
    steps=[
        (
            "col_transformer",
            column_transformer,
        ),
        (
            "custom_features",
            FunctionTransformer(lrmfp_features, validate=False),
        ),
    ]
)

df_custom = retail_pipeline.fit_transform(df_retail)

In [ ]:
explore_data(df_custom)

In [ ]:
df_custom.describe()

Las nuevas variables solo monetary tiene valores nulos. Utilizando este df, se ve que las variables precio y quantity tienen distribuciones más uniformes en sus límites y no tan concentradas en los valores bajos. Viendo la distribución de los datos la periodicidad también es un buen número por lo que la empresa tiene clientes fieles, lo que se confirma con el valor promedio de lenght. Si bien para todos estos casos la desviación estándar es alta, los promedios son un buen indicio para estudiar todo esto. El caso de frecuency es algo distinto, ya que su valor máximo está por sobre la media por lo que el promdio es poco representativo, quiere decir que no se puede explicar de manera simple el comportamiento del número total de visitas de cada cliente de manera agrupada.



### 5. MinMax Scaler [10 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [19]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.min_ = X.min()
        self.max_ = X.max()
        return self

    def transform(self, X):
        X_normalized = (X - self.min_) / (self.max_ - self.min_)
        return X_normalized

    def set_output(self, transform="default"):
        # No modificar este método
        return self


#### 5.2 Incorporando MinMax al pipeline [5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [1 punto]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [1 punto]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [1 punto]
- Reportar los cambios observados en la distribución de las variables.  [2 puntos]

>Nota: Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [20]:
numeric_transformations = Pipeline(
    steps=[
        ("iqr", IQR(lambd=2)),
        (
            "mean_imputer",
            SimpleImputer(strategy="mean"),
        ),
        ("minmax", MinMax()),
    ]
)

In [ ]:
numerical_columns = ["Price", "Quantity"]
categorical_columns = ["Invoice", "StockCode", "Customer ID", "Country", "InvoiceDate"]

column_transformer = ColumnTransformer(
    transformers=[
        ("numerical", numeric_transformations, numerical_columns),
        ("categorical", "passthrough", categorical_columns),
    ],
    verbose_feature_names_out=False,
)

column_transformer.set_output(transform="pandas")


In [22]:
df_minmax = column_transformer.fit_transform(df_retail)

In [ ]:
explore_data(df_retail)
explore_data(df_minmax)

Con la aplicación de MaxMin, se puede interpretar mejor los datos porque la distribución es más uniforme, no se consideran los outliers y permite trabajar de manera escalable con los datos, ya que todos los datos se mantienen entre 0 y 1.

### 6. Pregunta teórica [5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

Finalmente, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

*Escriba su respuesta aquí*

Aplicar pipelines tiene varios beneficios, en primer lugar, ordena y estandariza el código porque si bien todos estos cambios se pueden aplicar de manera directa a cada df, el aplicar pipelines permite mantener una estructura. Lo que por consecuencia, ayuda a la replicabilidad del código, ya que disminuye los errores humanos que puede llevar la programación. Además, facilita la implementación de procesos de ML, ya que estar aplicando los cambios de manera manual constantemente es agotador y puede provocar errores que conducen a malas conclusiones por el mal manejo de los datos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.



<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>